# 2D Elastic Wave PINN Forward Problem Validation

## 概要

本ノートブックは、Phase 2（pinn-2d-fdtd-integration）で実装された2D弾性波PINNコンポーネントを用いて、順問題の完全なワークフローを実証します。

### 目的

- FDTDデータを用いた2D PINN訓練の実行
- 訓練中の損失項（L_data, L_pde, L_bc）モニタリング
- R²スコアによる定量評価
- 2D波動場の空間分布可視化
- 誤差分布の空間解析

### 前提条件

- Phase 2実装（pinn-2d-fdtd-integration）が完了していること
- `/PINN_data/`に.npzファイル（最低2ファイル）が配置されていること
- GPU環境（CUDA 12.4対応）推奨

### 実行環境

- Python 3.11+
- PyTorch 2.4.0 with CUDA 12.4
- DeepXDE 1.15.0

## Step 0: セットアップとインポート

Phase 2実装のServiceクラスをimportし、再現性を確保するためにrandom seedを設定します。

In [ ]:
# Standard library imports
import sys
from pathlib import Path
import time

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

# PINN module imports (Phase 2 implementation)
from pinn.data.fdtd_loader import FDTDDataLoaderService
from pinn.data.dimensionless_scaler import CharacteristicScales, DimensionlessScalerService
from pinn.models.pinn_model_builder_2d import PINNModelBuilder2DService
from pinn.models.pde_definition_2d import PDEDefinition2DService
from pinn.training.train_2d import TrainingPipelineService
from pinn.training.callbacks import LossLoggingCallback, R2ValidationCallback, DivergenceDetectionCallback
from pinn.validation.r2_score import R2ScoreCalculator
from pinn.validation.plot_generator import PlotGeneratorService
from pinn.utils.seed_manager import SeedManager

# Configure plotting
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 10)

# Set random seed for reproducibility (Requirement 7.4)
SEED = 42
SeedManager.set_seed(SEED)
print(f"Random seed set to: {SEED}")

# Check GPU availability (Requirement 2.9)
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: GPU not available, training will be slow on CPU")

## Step 1: FDTDデータの読み込みと無次元化

このステップでは、`/PINN_data/`ディレクトリからFDTDシミュレーションデータを読み込み、無次元化を適用します。

### タスク2.1: データファイル読み込み
- FDTDデータファイルの検索と読み込み
- データディレクトリの存在確認（Requirement 1.7）
- データセットサイズの表示（Requirement 1.2）

### タスク2.2: 無次元化と変数範囲表示
- 特性スケールの計算（Requirement 1.3）
- 無次元化後の変数範囲表示（Requirement 1.4）

### タスク2.3: Train/Val分割
- 訓練データとバリデーションデータの分割（Requirement 1.5）

In [ ]:
# Task 2.1: FDTD data file loading (Requirements 1.1, 1.2, 1.7)
# Data directory
data_dir = Path("/PINN_data")

# Check if data directory exists (Requirement 1.7)
if not data_dir.exists():
    raise FileNotFoundError(
        f"Data directory not found: {data_dir}\n"
        "Please create /PINN_data/ and place FDTD .npz files there.\n"
        "Expected naming: p{pitch_um}_d{depth_um}.npz (e.g., p1250_d100.npz)"
    )

# Load FDTD data files (Requirement 1.1)
npz_files = sorted(data_dir.glob("p*_d*.npz"))
if len(npz_files) < 2:
    raise ValueError(
        f"Insufficient data files: {len(npz_files)} found, need at least 2.\n"
        f"Please add FDTD .npz files to {data_dir}"
    )

print(f"Found {len(npz_files)} FDTD data files:")
for f in npz_files:
    print(f"  - {f.name}")

# Create FDTD data loader (Phase 2 Service)
loader = FDTDDataLoaderService(data_dir=data_dir)

# Load elastic constants (for dimensionless scaling)
elastic_lambda = 51.2e9  # Pa (Aluminum 6061-T6)
elastic_mu = 26.1e9      # Pa
density = 2700.0         # kg/m³

# Load first file to estimate displacement scale
sample_data = loader.load_file(npz_files[0])
U_ref = np.std(np.concatenate([sample_data.Ux, sample_data.Uy]))

print(f"\nEstimated U_ref: {U_ref:.2e} m")

# Task 2.2: Create characteristic scales (Requirement 1.3)
scales = CharacteristicScales.from_physics(
    domain_length=0.04,  # 40mm
    elastic_lambda=elastic_lambda,
    elastic_mu=elastic_mu,
    density=density,
    displacement_amplitude=U_ref
)

# Create scaler
scaler = DimensionlessScalerService(scales)

print(f"\nCharacteristic scales:")
print(f"  L_ref = {scales.L_ref:.4f} m")
print(f"  T_ref = {scales.T_ref:.2e} s")
print(f"  U_ref = {scales.U_ref:.2e} m")
print(f"  σ_ref = {scales.sigma_ref:.2e} Pa")
print(f"  c_l   = {scales.velocity_ref:.0f} m/s")

# Load and normalize data (Requirement 1.2)
start_time = time.time()
dataset = loader.load_multiple_files(npz_files, apply_dimensionless=True, scaler=scaler)
load_time = time.time() - start_time

print(f"\n✓ Loaded {len(dataset.x)} samples from {len(npz_files)} file(s) in {load_time:.2f}s")

# Task 2.2: Display dimensionless variable ranges (Requirement 1.4)
print(f"\nData ranges (dimensionless):")
print(f"  x̃:  [{np.min(dataset.x):.3f}, {np.max(dataset.x):.3f}]")
print(f"  ỹ:  [{np.min(dataset.y):.3f}, {np.max(dataset.y):.3f}]")
print(f"  t̃:  [{np.min(dataset.t):.3f}, {np.max(dataset.t):.3f}]")
print(f"  T̃1: [{np.min(dataset.T1):.3f}, {np.max(dataset.T1):.3f}]")
print(f"  T̃3: [{np.min(dataset.T3):.3f}, {np.max(dataset.T3):.3f}]")
print(f"  Ũx: [{np.min(dataset.Ux):.3f}, {np.max(dataset.Ux):.3f}]")
print(f"  Ũy: [{np.min(dataset.Uy):.3f}, {np.max(dataset.Uy):.3f}]")

# Task 2.3: Train/val split (Requirement 1.5)
train_data, val_data = loader.train_val_split(dataset, train_ratio=0.8, seed=SEED)
print(f"\nTrain samples: {len(train_data.x)}")
print(f"Val samples: {len(val_data.x)}")

In [ ]:
# Task 2.3: Visualize spatiotemporal distribution (Requirement 1.6)
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(dataset.t, dataset.x, s=0.5, alpha=0.3, c='blue')
ax.set_xlabel("t̃ (dimensionless time)", fontsize=12)
ax.set_ylabel("x̃ (dimensionless x-coordinate)", fontsize=12)
ax.set_title("Spatiotemporal Sampling Distribution", fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Data preparation complete (Tasks 2.1, 2.2, 2.3)")

## Step 2: 2D PINNモデルの構築

このステップでは、Phase 2のPINNModelBuilder2DServiceを使用して2D PINNモデルを構築します。

### タスク3.1: 設定パラメータ定義とモデル構築
- 設定パラメータの定義（Requirement 2.1）
- 5D入力、4D出力のPINNモデル構築（Requirement 2.2）
- PDEDefinition2DServiceのPDE関数適用（Requirement 2.3）
- API整合性確保（Requirement 8.3）

**推奨実行時間**: 約10-30秒（モデル構築のみ、訓練は次ステップ）

In [ ]:
# Task 3.1: Configuration parameters definition (Requirement 2.1)
# Define PINN configuration parameters using Python dictionary format

config = {
    "network": {
        # 5D input: (x̃, ỹ, t̃, pitch_norm, depth_norm) (Requirement 2.2)
        # 4D output: (T̃1, T̃3, Ũx, Ũy)
        "layer_sizes": [5, 64, 64, 64, 4],
        "activation": "tanh"  # tanh activation for smooth approximation
    },
    "training": {
        "epochs": 5000,  # Minimum 1000, recommended 5000 (Requirement 2.4)
        "learning_rate": 0.001,  # Initial learning rate
        "loss_weights": {
            "data": 1.0,   # Weight for data fitting loss (L_data)
            "pde": 1.0,    # Weight for PDE residual loss (L_pde)
            "bc": 0.0      # Weight for boundary condition loss (L_bc)
        }
    }
}

print("Configuration parameters:")
print(f"  Network architecture: {config['network']['layer_sizes']}")
print(f"  Input dimension: {config['network']['layer_sizes'][0]} (x̃, ỹ, t̃, pitch_norm, depth_norm)")
print(f"  Output dimension: {config['network']['layer_sizes'][-1]} (T̃1, T̃3, Ũx, Ũy)")
print(f"  Hidden layers: {config['network']['layer_sizes'][1:-1]}")
print(f"  Activation: {config['network']['activation']}")
print(f"  Epochs: {config['training']['epochs']}")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Loss weights: L_data={config['training']['loss_weights']['data']}, L_pde={config['training']['loss_weights']['pde']}, L_bc={config['training']['loss_weights']['bc']}")

# Verify layer dimensions (Requirement 2.2)
assert config["network"]["layer_sizes"][0] == 5, "Input dimension must be 5"
assert config["network"]["layer_sizes"][-1] == 4, "Output dimension must be 4"
print("\n✓ Configuration validated")

# Task 3.1: Model construction (Requirements 2.2, 8.3)
print("\nConstructing PINN model...")

# Create model builder service (Phase 2 implementation)
builder = PINNModelBuilder2DService()

# Create PDE definition service for physics constraints (Requirement 2.3)
pde_service = PDEDefinition2DService(
    elastic_lambda=elastic_lambda,
    elastic_mu=elastic_mu,
    density=density,
    scales=scales
)

# Build 2D PINN model with 5D input, 4D output
# This creates a DeepXDE model with the specified architecture
model = builder.build_model(config)

print("✓ PINN model constructed successfully")
print(f"  Model type: {type(model)}")

# Get PDE function for physics constraints (Requirement 2.3)
pde_function = pde_service.create_pde_function()

print("✓ PDE function created for physics constraints")
print(f"  PDE type: Dimensionless 2D elastic wave equation")

# Verify model construction
assert model is not None, "Model construction failed"
print("\n✓ Model construction complete (Task 3.1)")

## Step 3: 訓練実行と損失監視

このステップでは、TrainingPipelineServiceを使用してPINN訓練を実行し、損失項をリアルタイムで監視します。

### タスク4.1: 訓練実行と損失ロギング
- LossLoggingCallback、R2ValidationCallback、DivergenceDetectionCallbackの設定
- TrainingPipelineServiceによる訓練実行（Requirements 2.4, 2.5）
- 訓練時間の記録（GPU使用時）（Requirement 2.9）
- 個別損失項（L_data, L_pde, L_bc）とtotal lossの表示（Requirement 2.6）

### タスク4.2: 訓練履歴プロットとNaN検出
- PlotGeneratorServiceによる訓練履歴の4系列プロット（Requirement 2.7）
- DivergenceDetectionCallbackによるNaN loss検出（Requirement 2.8）
- 訓練収束性の視覚的確認

**推奨実行時間**: GPU: 約10-15分（5000 epochs）、CPU: 約60-90分

In [ ]:
# Task 4.1: Training execution and loss logging (Requirements 2.4, 2.5, 2.6, 2.9)

# Prepare validation data for R2ValidationCallback
# Validation input: (N_val, 5) [x, y, t, pitch_norm, depth_norm]
val_x = np.column_stack([
    val_data.x,
    val_data.y,
    val_data.t,
    val_data.pitch_norm,
    val_data.depth_norm
])

# Validation output: (N_val, 4) [T1, T3, Ux, Uy]
val_y = np.column_stack([
    val_data.T1,
    val_data.T3,
    val_data.Ux,
    val_data.Uy
])

print(f"Validation data prepared:")
print(f"  val_x shape: {val_x.shape}")
print(f"  val_y shape: {val_y.shape}")

# Create output directory for training artifacts
output_dir = Path.cwd().parent / "experiments" / "forward_validation"
output_dir.mkdir(parents=True, exist_ok=True)
print(f"\nOutput directory: {output_dir}")

# Configure callbacks (Requirement 2.5, 2.8)
callbacks = [
    LossLoggingCallback(log_interval=100),  # Log every 100 epochs (Requirement 2.6)
    R2ValidationCallback(
        val_x=val_x,
        val_y=val_y,
        r2_threshold=0.9,
        log_interval=1000  # Compute R² every 1000 epochs
    ),
    DivergenceDetectionCallback(
        output_dir=output_dir,
        nan_threshold=1e10  # Halt if loss exceeds threshold (Requirement 2.8)
    )
]

print("\n✓ Callbacks configured:")
print("  - LossLoggingCallback (log_interval=100)")
print("  - R2ValidationCallback (r2_threshold=0.9, log_interval=1000)")
print("  - DivergenceDetectionCallback (nan_threshold=1e10)")

# Create training pipeline service
training_pipeline = TrainingPipelineService()

# Record training start time (Requirement 2.9)
print(f"\nStarting training with {config['training']['epochs']} epochs...")
print(f"GPU available: {torch.cuda.is_available()}")
training_start_time = time.time()

# Execute training (Requirement 2.4)
trained_model, history = training_pipeline.train(
    model=model,
    config=config,
    output_dir=output_dir,
    callbacks=callbacks
)

# Record training end time (Requirement 2.9)
training_end_time = time.time()
training_duration = training_end_time - training_start_time

print(f"\n{'='*60}")
print(f"Training Complete")
print(f"{'='*60}")
print(f"Training time: {training_duration:.2f} seconds ({training_duration/60:.2f} minutes)")
print(f"Epochs completed: {config['training']['epochs']}")
print(f"Final losses:")
if history:
    if "L_data" in history and len(history["L_data"]) > 0:
        print(f"  L_data: {history['L_data'][-1]:.6e}")
    if "L_pde" in history and len(history["L_pde"]) > 0:
        print(f"  L_pde: {history['L_pde'][-1]:.6e}")
    if "L_bc" in history and len(history["L_bc"]) > 0:
        print(f"  L_bc: {history['L_bc'][-1]:.6e}")
    if "total_loss" in history and len(history["total_loss"]) > 0:
        print(f"  Total: {history['total_loss'][-1]:.6e}")

print("\n✓ Training execution complete (Task 4.1)")

In [ ]:
# Task 4.2: Training history plot and convergence check (Requirement 2.7)

# Create plot generator service
plot_generator = PlotGeneratorService()

# Plot training curves with 4 series: L_data, L_pde, L_bc, Total loss (Requirement 2.7)
print("Generating training history plot...")

fig, ax = plot_generator.plot_training_curves(
    history=history,
    save_path=output_dir / "training_history.png",
    log_scale=False  # Linear scale for better visual interpretation
)

plt.tight_layout()
plt.show()

print(f"✓ Training history plot saved to: {output_dir / 'training_history.png'}")

# Visual convergence check (Requirement 2.7)
print("\nTraining convergence analysis:")
if history and "total_loss" in history and len(history["total_loss"]) > 100:
    initial_loss = history["total_loss"][0]
    final_loss = history["total_loss"][-1]
    loss_reduction = (initial_loss - final_loss) / initial_loss * 100
    
    print(f"  Initial total loss: {initial_loss:.6e}")
    print(f"  Final total loss: {final_loss:.6e}")
    print(f"  Loss reduction: {loss_reduction:.2f}%")
    
    if loss_reduction > 50:
        print("  ✓ Good convergence: Loss reduced by >50%")
    elif loss_reduction > 20:
        print("  ⚠ Moderate convergence: Loss reduced by >20% but <50%")
        print("    Recommendation: Consider increasing epochs or adjusting learning rate")
    else:
        print("  ✗ Poor convergence: Loss reduced by <20%")
        print("    Recommendation: Check loss weights, learning rate, or network architecture")
else:
    print("  Warning: Insufficient training history for convergence analysis")

# Check if DivergenceDetectionCallback detected any issues (Requirement 2.8)
divergence_diagnostic_path = output_dir / "divergence_diagnostic.json"
if divergence_diagnostic_path.exists():
    print("\n⚠ WARNING: NaN loss detected during training!")
    print(f"  Diagnostic file: {divergence_diagnostic_path}")
    print("  Recommendations:")
    print("    - Reduce learning rate (try 0.0001 instead of 0.001)")
    print("    - Adjust loss weights (try w_pde=0.1 if PDE loss dominates)")
    print("    - Check input data normalization")
else:
    print("\n✓ No NaN loss detected - training was stable")

print("\n✓ Training history visualization complete (Task 4.2)")